# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298388314776                   -0.85    5.0         
  2   -8.300198149282       -2.74       -1.26    1.1    166ms
  3   -8.300433529638       -3.63       -1.89    3.2    131ms
  4   -8.300461543399       -4.55       -2.78    1.6    103ms
  5   -8.300464358038       -5.55       -3.11    3.1    142ms
  6   -8.300464512324       -6.81       -3.30    9.2    207ms
  7   -8.300464577884       -7.18       -3.45    1.5    127ms
  8   -8.300464615532       -7.42       -3.61    2.4    166ms
  9   -8.300464635041       -7.71       -3.78    1.8    123ms
 10   -8.300464640151       -8.29       -3.91    1.0    101ms
 11   -8.300464643663       -8.45       -4.23    1.9    127ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63211427955                   -0.70    5.6         
  2   -16.63626273559       -2.38       -1.14    1.0    240ms
  3   -16.67910262441       -1.37       -1.72    2.6    283ms
  4   -16.67927200233       -3.77       -2.23    3.6    342ms
  5   -16.67928470230       -4.90       -2.85    3.2    332ms
  6   -16.67928614814       -5.84       -3.15    4.9    380ms
  7   -16.67928619819       -7.30       -3.30    1.1    230ms
  8   -16.67928621828       -7.70       -3.80    1.0    234ms
  9   -16.67928622137       -8.51       -4.53    2.9    293ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32585959240                   -0.56    7.1         
  2   -33.33272895384       -2.16       -1.00    1.2    931ms
  3   -33.33407144206       -2.87       -1.74    4.5    1.17s
  4   -33.33428697210       -3.67       -2.63    2.5    1.05s
  5   -33.33693979679       -2.58       -2.49    5.6    1.48s
  6   -33.33694059023       -6.10       -2.51    7.6    1.26s
  7   -33.33694322979       -5.58       -2.97    1.9    875ms
  8   -33.33694354219       -6.51       -3.45    3.6    1.00s
  9   -33.33694374566       -6.69       -3.62    3.6    1.14s
 10   -33.33694377884       -7.48       -4.15    1.5    912ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298543892516                   -0.85    5.1         
  2   -8.300245715542       -2.77       -1.58    1.1   81.2ms
  3   -8.300186582129   +   -4.23       -2.03    2.8    112ms
  4   -8.300377723771       -3.72       -2.27    1.6   89.2ms
  5   -8.300463750825       -4.07       -3.49    1.1   81.0ms
  6   -8.300464498720       -6.13       -3.66    6.8    184ms
  7   -8.300464593134       -7.02       -3.89    2.4    128ms
  8   -8.300464641696       -7.31       -4.55    1.0   91.4ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32572610707                   -0.56    7.0         
  2   -33.31825994640   +   -2.13       -1.27    1.0    790ms
  3   -17.60458034411   +    1.20       -0.47    5.9    1.63s
  4   -33.32340125215        1.20       -1.74    5.9    1.54s
  5   -33.22572197231   +   -1.01       -1.31    3.2    1.36s
  6   -33.26191666865       -1.44       -1.62    3.4    1.23s
  7   -33.21447489287   +   -1.32       -1.52    4.0    1.24s
  8   -33.33633520033       -0.91       -2.42    3.1    1.06s
  9   -33.33672884934       -3.40       -2.83    4.1    1.34s
 10   -33.33692396547       -3.71       -3.18    3.4    1.12s
 11   -33.33689575741   +   -4.55       -2.96    2.8    1.27s
 12   -33.33694027205       -4.35      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.